### I. Comparing Python Classes and Extension Types
Python中一切皆对象。在最基础的级别，对象有三要素：identity、value、type。
- identity：由内建的<font face='consolas'>id</font>函数给出，用以区分其他对象。
- value：其关联的数据，由"."来访问。通常Python将对象的数据放在内部的instance dictionary <font face='consolas'>__dict__</font>中。
- type：指明该type对象的行为，行为由函数给出，即为method。

使用Python/C API直接创的type称为<font color='#ff2222'>extension types</font>，远比Python class关键字创建的type要高效，但是需要精通Python/C API才行。

Cython可以通过<font color='#ff2222' face='consolas'>cdef class</font>以和Python相似的语法创建extension type。
- cdef class语句告诉Cython创建extenstion type
- 类体重的cdef类型声明是<font color='#ff2222'>C级别的instance属性</font>而不是class属性。
    - <font color='#ff2222'>Extension type的属性默认为private，只能被类的方法访问。</font>下面的例子中可以看到直接通过cy_particle.mass报错了。
    - 当extension type被初始化后，一个C struct被分配和初始化，其是固定的，不对新成员开放，所以不能动态绑定新属性。而Python类属性保存在instance的__dict__中，字典是可变的，所以可以动态绑定新属性
    
### II. Type Attribute and Access Control
Python的查询机制适用于instance属性、方法、基础类的属性等，而适用范围广度的代价是性能开销。Cython的cdef class extension type则绕过了该查询机制，这极大的提高了性能。

我们可以通过一些关键字来控制Cython instance属性的访问级别。调用试用属性的方法时，这些关键字将被忽略。
- <font face='consolas'>readonly</font>: 只读属性
- <font face='consolas'>public</font>: 可读可写


In [1]:
%load_ext Cython

In [2]:
%%file python_particle.py
class Particle(object):
    '''Simple Particle type.'''
    def __init__(self, m, p, v):
        self.mass = m
        self.position = p
        self.velocity = v
    def get_momentum(self):
        return self.mass * self.velocity

Writing python_particle.py


In [5]:
%%file cython_particle.pyx
cdef class Particle:
    '''Simple Particle type.'''
    cdef public double mass
    cdef readonly double position
    cdef double velocity
    def __init__(self, m, p, v):
        self.mass = m
        self.position = p
        self.velocity = v
    def get_momentum(self):
        return self.mass * self.velocity

Overwriting cython_particle.pyx


In [6]:
import pyximport
pyximport.install()
import cython_particle
import python_particle

In [7]:
py_particle = python_particle.Particle(1.0, 2.0, 3.0)
cy_particle = cython_particle.Particle(1.0, 2.0, 3.0)

In [8]:
print py_particle.get_momentum()
print cy_particle.get_momentum()
print py_particle.mass, py_particle.position, py_particle.velocity
print cy_particle.mass, cy_particle.position
cy_particle.mass = 10.0
print cy_particle.mass
cy_particle.velocity=10.0

3.0
3.0
1.0 2.0 3.0
1.0 2.0
10.0


AttributeError: 'cython_particle.Particle' object has no attribute 'velocity'

### III. C-Level Initialization and Finalization
当Python调用\_\_init\_\_时,self参数必须是一个有效的extension type instance。当\_\_init\_\_被调用时，通常其会初始化self参数的属性。在C级别，\_\_init\_\_被调用之前，必须为instance的struct分配内存，所有的struct field必须处于有效的状态，准备接收初始值。

前例中的Particle extension type中，\_\_init\_\_扮演了这个角色，因为所有的field都是double类型，不需要C级别的内存分配。但是根据extension type 怎样被subclassed，或者有其他构造器，\_\_init\_\_在对象创建过程中可能被调用多次，从而导致内存泄露；而其他一些情况则可能导致\_\_init\_\_被完全跳过，而导致丑陋的segmentation faults。Cython添加了一个特殊的方法\_\_cinit\_\_负责执行C级别的内存分配和初始化，其只执行一次，且在\_\_init\_\_，\_\_new\_\_，及其他Python-level构造器之前被执行。

Cython通过\_\_dealloc\_\_方法执行C-level finalization，执行\_\_cinit\_\_的反向操作，其也保证只执行一次。

In [9]:
%%cython
from libc.stdlib cimport malloc, free
cdef class Matrix:
    cdef:
        unsigned int nrows, ncols
        double *_matrix
    def __cinit__(self, nr, nc):
        self.nrows = nr
        self.ncols = nc
        # malloc分配内存，若成功返回被分配内存的指针
        self._matrix = <double*>malloc(nr * nc * sizeof(double))
        if self._matrix == NULL:
            raise MemoryError()
    def __dealloc__(self):
        if self._matrix != NULL:
            free(self._matrix)

### IV. cdef and cpdef Methods
cdef和cpdef定义的method不可以用在non-cdef类中。
用法不再赘述，参见Chapter 3。
<font color='#ff2222'>page 106</font>

### V. Inheritance and Subclassing
extension type可以单继承，不可多继承。base type必须由C实现——内建类型或其他extension type，不能是一般的Python类，否则会导致cython compile-time error。

In [25]:
%%file setup.py
from distutils.core import setup
from distutils.extension import Extension
from Cython.Build import cythonize
setup(ext_modules=cythonize('cython_particle.pyx'))

Writing setup.py


In [28]:
!python setup.py build_ext --inplace --compiler=msvc

running build_ext
building 'cython_particle' extension
creating build
creating build\temp.win-amd64-2.7
creating build\temp.win-amd64-2.7\Release
C:\Users\Think X260\AppData\Local\Programs\Common\Microsoft\Visual C++ for Python\9.0\VC\Bin\amd64\cl.exe /c /nologo /Ox /MD /W3 /GS- /DNDEBUG "-ID:\Program Files\Anaconda\include" "-ID:\Program Files\Anaconda\PC" /Tccython_particle.c /Fobuild\temp.win-amd64-2.7\Release\cython_particle.obj
cython_particle.c
C:\Users\Think X260\AppData\Local\Programs\Common\Microsoft\Visual C++ for Python\9.0\VC\Bin\amd64\link.exe /DLL /nologo /INCREMENTAL:NO "/LIBPATH:D:\Program Files\Anaconda\libs" "/LIBPATH:D:\Program Files\Anaconda\PCbuild\amd64" /EXPORT:initcython_particle build\temp.win-amd64-2.7\Release\cython_particle.obj "/OUT:D:\Program Files\Anaconda\QieLearning\Cython\Chapter 5 Cython and Extension Types\cython_particle.pyd" /IMPLIB:build\temp.win-amd64-2.7\Release\cython_particle.lib /MANIFESTFILE:build\temp.win-amd64-2.7\Release\cython_particle.p

In [18]:
%%cython
'''
为什么从cython_particle中导入的Particle类继承会报错？
'''
cdef class Particle:
    '''Simple Particle type.'''
    cdef public double mass
    cdef readonly double position
    cdef double velocity
    def __init__(self, m, p, v):
        self.mass = m
        
        self.position = p
        self.velocity = v
    def get_momentum(self):
        return self.mass * self.velocity

cdef class CParticle(Particle):
    cdef double momentum
    def __init__(self, m, p, v):
        super(Particle,self).__init__(m, p, v)
        self.momentum = self.mass * self.velocity
    cpdef double get_momentum(self):
        return self.momentum

### VI. Casting and Subclasses
当我们work with a dynamically typed object时，Cython无法访问其C-level data及方法。所有的属性查询都必须通过Python/C API,速度很慢。当我们知道动态变量是或者可能是一个内建类型或extension type时，将其转换为静态类型是值得的。这使Cython可以访问C-level属性和方法。更进一步的，Cython也可以不通过Python/C API，直接访问Python-level的属性和cpdef方法。

实现上述类型转换有两种方法：
- 创建一个我们想要的静态类型的变量，并将动态变量赋给它。
- 使用Cython的类型转换操作符，参见Chapter3

In [28]:
p = Particle(1.0, 2.0, 3.0)
%%cython
cdef Particle static_p = p
print static_p.get_momemtum()
print static_p.velocity
# 使用casting operator
print (<Particle>p).get_momentum()
print (<Particle>p).velocity

SyntaxError: invalid syntax (<ipython-input-28-3da4b3f10649>, line 2)

### VII. Extension Type Objects and None
page 109
noncheck compiler directive使所有的函数和method都是None-safe的。
> \# cython: nonecheck=True
> cython--directive nonecheck=True source.pyx

### VIII. Extension Type Properties in Cython
getter和setter方法：
- \_\_get\_\_(self)
- \_\_set\_\_(self, set_value)

In [29]:
def dispatch(Particle p):
    if p is None:
        raise TypeError("...")
    print p.get_momentum()
    print p.velocity

# Cython提供了特殊语法以实现上述操作
def dispatch2(Particle p not None):
    print p.get_momentum()
    print p.velocity

SyntaxError: invalid syntax (<ipython-input-29-1694eeed1835>, line 1)

### X. Special Methods Are Even More Special
特殊方法是指双下划线开头和结尾的方法，譬如\_\_cinit\_\_，\_\_init\_\_，\_\_dealloc\_\_等。
#### 算数方法Arithmetic Methods
为了让一个纯Python类C支持in-place加法，我们定义<font face='consolas'>\_\_add\_\_(self, other)</font>方法。<font face='consolas'>c + d</font>被转化为<font face='consolas'>C.\_\_add\_\_(c,d)</font>，其中<font face='consolas'>c</font>是C类的实例。如果C不知道怎么将自己与其他参数相加，返回<font face='consolas'>NotImplemented</font>。在这种情况下，Python解释器调用<font face='consolas'>type(d).\_\_radd\_\_(d, c)</font>给<font face='consolas'>b</font>的类一个机会，将其自身和C的实例相加。

对于extension types，情况有些不同。Extension types不支持<font face='consolas'>\_\_radd\_\_</font>，它们重载<font face='consolas'>\_\_add\_\_</font>方法，在一个方法中实现<font face='consolas'>\_\_add\_\_</font>和<font face='consolas'>\_\_radd\_\_</font>的功能。——<font color='#ff2222'>page 112</font>

In [34]:
%%file special_methods.pyx
cdef class E:
    """Extension type that supports addition."""
    cdef int data
    def __init__(self, d):
        self.data = d
    def __add__(x, y):
        # Regular __add__ behavior
        if isinstance(x, E):
            if isinstance(y, int):
                return (<E>x).data + y
        # __radd__ behavior
        elif isinstance(y, E):
            if isinstance(x, int):
                return (<E>y).data + x
        else:
            return NotImplemented

Overwriting special_methods.pyx


In [38]:
import pyximport
pyximport.install()
import special_methods
e = special_methods.E(100)
# __add__
print e + 1
# __radd__
print 1 + e
# NotImplemented
print e + 1.0
print 1.0 + e

101
101
None
None


#### Rich Comparisons
Cython extension types不支持用于个体比较的特殊方法，譬如<font face='consolas'>\_\_eq\_\_，\_\_lt\_\_，\_\_le\_\_</font>等。Cython提供一个方法<font face='consolas'>\_\_richcmp\_\_(x, y, op)</font>实现这些比较操作，传入的第三个整数参数op指明执行的比较操作。下表中的整数参数是在Python runtime object.h头文件中声明的compile-time constants。我们可以通过cimport访问这些常量（<font color='#ff2222'>详见Chapter 6</font>）

> from cpython.object cimport Py_LT, Py_LE, Py_EQ, Py_GE, Py_GT, Py_NE

|Integer argument|Comparison|
|:----:|:----:|
|Py_LT |  <  |
|Py_LE |  <=  |
|Py_EQ |  ==  |
|Py_NE |  !=  |
|Py_GT |  >  |
|Py_GE |  <=  |

In [39]:
%%file rich_comparisons.pyx
from cpython.object cimport Py_LT, Py_LE, Py_EQ, Py_GE, Py_GT, Py_NE
cdef class R:
    """Extension type that supports rich comparisons."""
    cdef double data
    def __init__(self, d):
        self.data = d
    def __richcmp__(x, y, int op):
        cdef:
            R r
            double data
        # Make r always refer to the R instance.
        r, y = (x, y) if isinstance(x, R) else (y, x)
        data = r.data
        if op == Py_LT:
            return data < y
        elif op == Py_LE:
            return data <= y
        elif op == Py_EQ:
            return data == y
        elif op == Py_NE:
            return data != y
        elif op == Py_GT:
            return data > y
        elif op == Py_GE:
            return data >= y
        else:
           assert False

Writing rich_comparisons.pyx


In [42]:
import pyximport
pyximport.install()
from rich_comparisons import R
r = R(10)
print r < 20 and 20 > r
print r > 20 and 20 < r
print 0 <= r <= 100
print r == 10
print r != 10
print r == 20
print 20 == r


True
False
True
True
False
False
False


#### Iterator Support
为了使一个extension type可被迭代，我们为其定义<font face='consolas'>\_\_iter\_\_</font>方法。为了使一个extension type成为一个迭代器，我们定义我们为其定义<font face='consolas'>\_\_next\_\_</font>方法。Cython会将<font face='consolas'>\_\_next\_\_</font>以next暴露给Python。

In [43]:
%%file iter_support.pyx
cdef class I:
    cdef:
        list data
        int i
    def __init__(self):
        self.data = range(100)
        self.i = 0
    def __iter__(self):
        return self
    def __next__(self):
        if self.i >= len(self.data):
            raise StopIteration()
        ret = self.data[self.i]
        self.i += 1
        return ret

Writing iter_support.pyx


In [44]:
import pyximport
pyximport.install()
from iter_support import I
i = I()
s = 0
for x in i:
    s += x
print s
it = iter(I())
print it.next()
print next(it)

4950
0
1
